In [1]:
from gensim.models import Word2Vec, KeyedVectors

from wiki_analyzer.corpus_manager import SQLiteCorpus
from wiki_analyzer.config import (
    DB_PATH,
    WIKI_MODEL_PATH,
    WORD2_VEC_MODEL_PATH
)
wiki_model_path_str = str(WIKI_MODEL_PATH)

In [2]:
# コーパスを読み込む
corpus = SQLiteCorpus(DB_PATH)

# Word2Vecモデルの訓練
model = Word2Vec(
    sentences=corpus,
    vector_size=200,
    window=10,
    min_count=5,
    workers=4,
    sample=1e-3,
    negative=5,
    sg=1  # skip-gramの場合は1、CBOWの場合は0
)

# モデルを保存する
model.save(wiki_model_path_str)

Loading corpus:  39%|███▊      | 867425/2252319 [05:21<08:33, 2695.34it/s] 


KeyboardInterrupt: 

In [2]:
# モデルのロードとテスト
model = Word2Vec.load(wiki_model_path_str)

# モデルのバイナリ形式で保存
model.wv.save_word2vec_format(WORD2_VEC_MODEL_PATH, binary=True)

In [3]:
# モデルのキーを取得
wv = KeyedVectors.load_word2vec_format(WORD2_VEC_MODEL_PATH, binary=True, encoding='utf-8')

In [4]:
# 類似単語の取得
results = wv.most_similar(positive='定食', topn=10)
for result in results:
    print(result)

('カツ丼', 0.8861806988716125)
('とんかつ', 0.8640125393867493)
('豚カツ', 0.8552635312080383)
('トンカツ', 0.8394948244094849)
('チャーハン', 0.8373618721961975)
('しゃぶしゃぶ', 0.837273120880127)
('すき焼き', 0.836484968662262)
('和食', 0.8325155973434448)
('焼き鳥', 0.8317357897758484)
('丼物', 0.8311268091201782)


In [5]:
# 類似度計算
similarity = wv.similarity('牛丼', 'カレー')
print(similarity)

0.645333
